In [ ]:
import google.generativeai as genai
import streamlit as st
import requests
from bs4 import BeautifulSoup

# API 설정
YOUR_API_KEY = 'key'
genai.configure(api_key=YOUR_API_KEY)

st.title("상담")

@st.cache_resource
def load_model():
    """
    제미나이 모델을 로드합니다.
    """
    model = genai.GenerativeModel(
      model_name='gemini-1.5-chat',
      temperature=0.5,
      max_output_tokens=500)
    print("모델이 로드되었습니다.")
    return model

model = load_model()

# 대화 세션 초기화
if "chat_session" not in st.session_state:
    st.session_state["chat_session"] = genai.start_chat(
        model="gemini-1.5-chat",  # 모델 이름 지정
        temperature=0.5,         # 응답의 창의성 제어
        max_output_tokens=500,   # 응답의 최대 토큰 수
        history=[
            {"role": "system", "content": "당신은 심리적으로 공감하고 현실적인 대책을 궁리해주는 상담사입니다. 사용자에게 도움을 주세요."}
        ]
    )
# 대화 기록 표시
for content in st.session_state.chat_session.history:
    with st.chat_message("상담사" if content['role'] == "model" else "user"):
        st.markdown(content['content'])

# 사용자 입력 처리
if prompt := st.chat_input("메시지를 입력하세요."):
    with st.chat_message("user"):
        st.markdown(prompt)
    with st.chat_message("상담사"):
        response = st.session_state.chat_session.send_message(
        message=prompt,
        temperature=0.5,
        max_output_tokens=500
        )
        st.markdown(response['content'])

# 웹크롤링 함수
@st.cache_resource
def fetch_counseling_articles():
    """
    국립 정신건강 센터에서 심리상담 관련 정보 크롤링
    """
    url = ["https://nct.go.kr/distMental/rating/rating02_2.do",
            "https://www.mohw.go.kr/menu.es?mid=a10706040100",
            ""
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    print("심리적인 어려움을 혼자 해결하기 어렵다면 전문가와의 상담을 추천합니다.")
    print("국립정신건강센터 등 전문 기관에서 상담을 받아보세요.")
    print("더 자세한 정보는 다음 링크에서 확인할 수 있습니다:", url)

if __name__ == "__main__":
    recommend_professional_counseling()